In [7]:
#read data(exclude replies) 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            token = porter.stem(token) #root form
            doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

The number of tweets (excludes replies): 6501


In [26]:
#unigram
unigram = []
unigram_list = []
for index, i in text.iterrows():
    unigram=''
    for word in i['processed'].split():
        unigram+= word+' '
    unigram_list.append(unigram)


import guidedlda
import numpy as np
import gensim

seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss'], #Environmental Impact
                   ['plant','vegan','vegetarian','animal','without','slaughter',
                   'cruelti','welfar'],# Animal Welfare
                  ['antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul',
                  'demand','secur','wast','industri','suppli','convent','regular','system'], #Food System
                  ['seafood','sea','ocean','fish','aquacultur','shrimp'],#Seafood
                  ['delici','tast','tender','textur','juici','test','chicken','duck','poultri','egg'],#Poultry
                  ['cell','stem','cultur','muscl','biolog','divis'], #Process
                  ['seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit'], #Announcement
                  ['thank','thankyou','support','shoutout','help'], #Appreciation
                  ['confer','regist','live','symposium','stream','livestream'], #conference and summit
                  ['market','consum','store','groceri'], # Market
                  ['check','post','paper','interview','articl','blog','news','break'], #Media
                  ['hire','join','team']]#Hiring Information

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
model = guidedlda.GuidedLDA(n_topics=12,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(text['processed'])

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.35)
topic_word = model.topic_word_
n_top_words = 30
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end=' ')

X = X.toarray()
doc_topic = model.transform(X)
topic_number = []
number = []
for i in range(len(doc_topic)-1):
    topic_number.append(doc_topic[i].argmax())
    number.append('1')
data = pd.DataFrame(data=[i for i in topic_number],columns=['topic_number'])
data['number'] = [i for i in number]
print(data.groupby('topic_number')['number'].count())

INFO:guidedlda:n_documents: 6501
INFO:guidedlda:vocab_size: 3621
INFO:guidedlda:n_words: 60068
INFO:guidedlda:n_topics: 12
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -694392
INFO:guidedlda:<10> log likelihood: -490422
INFO:guidedlda:<20> log likelihood: -474945
INFO:guidedlda:<30> log likelihood: -467499
INFO:guidedlda:<40> log likelihood: -463902
INFO:guidedlda:<50> log likelihood: -461861
INFO:guidedlda:<60> log likelihood: -460037
INFO:guidedlda:<70> log likelihood: -458819
INFO:guidedlda:<80> log likelihood: -457591
INFO:guidedlda:<90> log likelihood: -456854
INFO:guidedlda:<100> log likelihood: -456434
INFO:guidedlda:<110> log likelihood: -455634
INFO:guidedlda:<120> log likelihood: -455670
INFO:guidedlda:<130> log likelihood: -455698
INFO:guidedlda:<140> log likelihood: -455183
INFO:guidedlda:<150> log likelihood: -455020
INFO:guidedlda:<160> log likelihood: -454832
INFO:guidedlda:<170> log likelihood: -454291
INFO:guidedlda:<180> log likelihood: -454387
INFO:g



Topic: 0
0.0271*chang 0.0255*food 0.0216*anim 0.0183*world 0.018*climat 0.0154*need 0.0144*sustain 0.0124*human 0.0116*solut 0.0113*health 0.0101*peopl 0.0095*produc 0.0092*environment 0.0088*global 0.0088*impact 0.0087*system 0.0087*agricultur 0.0085*reduc 0.0085*industri 0.0083*grow 0.0082*feed 0.0082*use 0.008*planet 0.0077*eat 0.0072*water 0.007*product 0.007*earth 0.007*problem 0.007*help 0.0069*would 

Topic: 1
0.0751*anim 0.0407*make 0.0366*without 0.0212*world 0.0208*real 0.0206*better 0.0206*free 0.0206*slaughter 0.0197*futur 0.0184*help 0.0167*want 0.016*need 0.0158*grow 0.0156*planet 0.0147*peopl 0.013*food 0.013*save 0.0113*kill 0.0113*love 0.01*produc 0.01*harm 0.0095*support 0.0087*vegan 0.0082*human 0.0082*check 0.0078*chang 0.0076*cow 0.0074*billion 0.0063*cruelti 0.0063*word 

Topic: 2
0.0413*food 0.0361*product 0.0221*plant 0.0212*industri 0.0189*market 0.0179*base 0.013*protein 0.0126*global 0.0123*scale 0.0096*suppli 0.0095*demand 0.0086*chain 0.0086*system 0.0086

0.0079*answer 0.0079*would 0.0077*chanc 0.0077*talk 

Topic: 8
0.027*speak 0.0222*join 0.0212*futur 0.0189*food 0.0179*confer 0.0179*panel 0.0154*discuss 0.0142*event 0.0138*th 0.0122*today 0.0113*summit 0.0105*present 0.0097*tomorrow 0.0093*live 0.0091*tune 0.0089*ceo 0.0089*next 0.0087*pitch 0.0076*come 0.0076*week 0.007*vote 0.007*meet 0.007*talk 0.0068*regist 0.0066*june 0.0066*scienc 0.0066*excit 0.0058*innov 0.0058*pm 0.0056*host 

Topic: 9
0.0749*grown 0.0565*lab 0.0514*steak 0.0411*farm 0.0396*first 0.0314*cell 0.0302*aleph 0.0225*startup 0.0225*world 0.0181*isra 0.0148*compani 0.0133*free 0.0133*beef 0.013*slaughter 0.0125*start 0.0125*produc 0.0123*rais 0.0123*creat 0.0087*israel 0.0082*didier 0.0082*space 0.0082*could 0.0079*toubia 0.0079*million 0.0077*made 0.0074*shape 0.0069*experi 0.0066*viand 0.0061*print 0.0061*readi 

Topic: 10
0.024*founder 0.0226*check 0.0214*featur 0.0214*great 0.0212*food 0.021*thank 0.0193*futur 0.0181*co 0.0171*interview 0.0163*talk 0.0149*artic